In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
from IPython.display import Image, display
from io import BytesIO
from PIL import Image


In [ ]:
data_path = r"C:\Users\avoav\OneDrive - Ariel University\year 3\final project\DATA_PHOTOS_IAA_NUMBERS"
# Load images using OpenCV
image_list = []
for filename in os.listdir(data_path):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.tif'):
        img = cv2.imread(os.path.join(data_path, filename))
        image_list.append(img)

In [ ]:
fig = plt.figure(figsize=(5, 6))
  
fig.add_subplot(1, 1, 1)
plt.imshow(image_list[0])

In [ ]:
# base_url = 'https://github.com/Aviost111/final_proj/tree/main/REAL_DATA/'
# start_number = 150006  # Starting number
# end_number = 151051  # Ending number
# num_images_per_group = 2  # Number of images per group (e.g., 150006.1.tif and 150006.2.tif)

# image_urls = []

# for i in range(start_number, end_number + 1):
#     for j in range(1, num_images_per_group + 1):
#         image_urls.append(f'{base_url}{i}.{j}.tif')


In [ ]:
# len(image_urls)

## First task

Isolate the coin from the image (remove the ruler and the note).

In [ ]:
def crop_coin_simple(img, output_path):

    # Convert the image to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the color range for the coin
    lower_bound = np.array([20, 50, 50])  # Adjust these values based on the coin's color
    upper_bound = np.array([30, 255, 255])

    # Create a mask using the color range
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Choose the largest contour (assumed to be the coin)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Find extreme points (leftmost, rightmost, topmost, bottommost) of the contour
    leftmost = tuple(largest_contour[largest_contour[:,:,0].argmin()][0])
    rightmost = tuple(largest_contour[largest_contour[:,:,0].argmax()][0])
    topmost = tuple(largest_contour[largest_contour[:,:,1].argmin()][0])
    bottommost = tuple(largest_contour[largest_contour[:,:,1].argmax()][0])

    # Define the region of interest (ROI) using the coordinates of the bounding box
    roi = img[topmost[1]:bottommost[1], leftmost[0]:rightmost[0]]

    # Save the cropped region
    cv2.imwrite(output_path, roi)
            

In [ ]:
def crop_images(output_dir , image_list):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Process each image in image_list
    for idx, img in enumerate(image_list):
        output_path = os.path.join(output_dir, f"cropped_image_{idx}.jpg")
        crop_coin_simple(img, output_path)
        #print(f"Image {idx} cropped and saved to {output_path}")


In [ ]:
crop_images("REAL_DATA_CROPPED1",image_list)

In [7]:
def add_padding(image, target_size):
    # Calculate padding dimensions
    h, w = image.shape[:2]
    target_h, target_w = target_size
    pad_h = max(0, target_h - h)
    pad_w = max(0, target_w - w)
    
    # Add padding to the image
    padded_image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    
    return padded_image 
    
# Load images using OpenCV
data_path = "REAL_DATA_CROPPED1"
image_list = []
max_height = 0
max_width = 0

# Determine maximum width and height
for filename in os.listdir(data_path):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.tif'):
        img = cv2.imread(os.path.join(data_path, filename))
        h, w = img.shape[:2]
        max_height = max(max_height, h)
        max_width = max(max_width, w)
        image_list.append(img)
       
# Add padding to each image
padded_image_list = []
for img in image_list:
    padded_img = add_padding(img, (max_height, max_width))
    padded_image_list.append(padded_img)

# Save padded images to a new directory
output_dir = "REAL_DATA_Padded"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, padded_img in enumerate(padded_image_list):
    output_path = os.path.join(output_dir, f"padded_image_{i}.jpg")
    cv2.imwrite(output_path, padded_img)


In [8]:
# Check if all images have the same size
sizes = set()
for img in padded_image_list:
    sizes.add(img.shape[:2])

if len(sizes) == 1:
    print("All images have the same size.")
    print("Size:", sizes.pop())
else:
    print("Images have different sizes.")


All images have the same size.
Size: (3122, 2983)


## combining two sides of each coin

In [4]:
# Function to combine two images
def combine_images(image1_path, image2_path, output_folder):
    # Load images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Resize images to have the same dimensions (optional, depends on your use case)
    image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

    # Combine images horizontally
    combined_image = cv2.hconcat([image1, image2])

    # Extract file names
    image1_name = os.path.basename(image1_path)
    image2_name = os.path.basename(image2_path)

    # Generate output file name
    output_filename = f"combined_{image1_name}_{image2_name}"

    # Save the combined image
    output_path = os.path.join(output_folder, output_filename)
    cv2.imwrite(output_path, combined_image)

# Path to your folder containing images
input_folder_path = r"C:\Users\avoav\OneDrive - Ariel University\year 3\final project\DATA_PHOTOS_IAA_NUMBERS"

# Path to the folder where combined images will be saved
output_folder_path = "Combined_images"

# Get a list of image files in the input folder
image_files = os.listdir(input_folder_path)

# Iterate through the image files to find pairs with names like "some.1" and "some.2"
for image_file in image_files:
    if image_file.endswith(".1"):
        image1_path = os.path.join(input_folder_path, image_file)
        image2_name = image_file.replace(".1", ".2")
        image2_path = os.path.join(input_folder_path, image2_name)
        if os.path.exists(image2_path):
            combine_images(image1_path, image2_path, output_folder_path)
